<a href="https://colab.research.google.com/github/helen-mark/phonecall/blob/master/MCP_Call_Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install zstd
!curl -fsSL https://ollama.com/install.sh | sh
!pip install ollama

!pip list | grep ollama

!ollama list

import subprocess, time
print("Запускаю Ollama сервер...")
subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(10)

import requests
try:
    response = requests.get("http://localhost:11434/api/tags", timeout=5)
    if response.status_code == 200:
        print("✅ Ollama сервер работает")
    else:
        print(f"⚠️  Ollama сервер отвечает с кодом {response.status_code}")
except:
    print("❌ Ollama сервер не отвечает")

print("Скачиваю модель mistral-nemo:12b...")
!ollama pull mistral-nemo:12b



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
zstd is already the newest version (1.4.8+dfsg-3build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
ollama                                   0.6.1
Error: ollama server not responding - could not connect to ollama server, run 'ollama serve' to start it
Запускаю Ollama сервер...
✅ Ollama сервер работает
Скачиваю модель mistral-nemo:12b...



In [ ]:
import os
# ====================
# 🚀 ЗАГРУЗКА КОДА ИЗ GITHUB РЕПОЗИТОРИЯ
# ====================

from google.colab import files
import sys

print("📦 Загружаю проект из GitHub...")

repo_url = "https://github.com/helen-mark/phonecall.git"  # 🚨 ЗАМЕНИТЕ ЭТО
repo_name = "phonecall"  # 🚨 ЗАМЕНИТЕ ЭТО

!rm -rf {repo_name} 2>/dev/null || true

!git clone {repo_url}

%cd {repo_name+ '/colab'}

print("\n📁 Структура проекта:")
!find . -name "*.py" | head -20

project_path = os.getcwd()
sys.path.insert(0, project_path)
print(f"\n📍 Проект загружен в: {project_path}")
print(f"📦 Python path обновлен")

data_dirs = ["csv_calls", "saved_results", "logs"]
for dir_name in data_dirs:
    os.makedirs(dir_name, exist_ok=True)
    print(f"✅ Создана директория: {dir_name}/")

required_files = ["main.py", "interactive.py", "mcp_orchestrator.py"]
missing_files = []

for file in required_files:
    if os.path.exists(file):
        print(f"✅ Найден: {file}")
    else:
        missing_files.append(file)
        print(f"❌ Отсутствует: {file}")

if missing_files:
    print(f"\n⚠️  Отсутствуют файлы: {missing_files}")
    print("Проверьте структуру репозитория на GitHub")
else:
    print("\n🎉 Все файлы загружены успешно!")

print("📤 ЗАГРУЗИТЕ CSV ФАЙЛЫ ЗВОНКОВ:")
csv_files = files.upload()

for filename in csv_files.keys():
    if filename.endswith('.csv'):
        os.rename(filename, f"/content/csv_calls/{filename}")
        print(f"✅ CSV файл: {filename}")



📦 Загружаю проект из GitHub...
Cloning into 'phonecall'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 82 (delta 31), reused 72 (delta 21), pack-reused 0 (from 0)
Receiving objects: 100% (82/82), 283.19 KiB | 8.33 MiB/s, done.
Resolving deltas: 100% (31/31), done.
/content/phonecall/colab

📁 Структура проекта:
./main.py
./colab_minimal.py
./colab_drive_setup.py
./interactive.py
./mcp_orchestrator.py

📍 Проект загружен в: /content/phonecall/colab
📦 Python path обновлен
✅ Создана директория: csv_calls/
✅ Создана директория: saved_results/
✅ Создана директория: logs/
✅ Найден: main.py
✅ Найден: interactive.py
✅ Найден: mcp_orchestrator.py

🎉 Все файлы загружены успешно!
📤 ЗАГРУЗИТЕ CSV ФАЙЛЫ ЗВОНКОВ:


Saving calls.csv to calls.csv
✅ CSV файл: calls.csv


In [ ]:
import sys
sys.path.append('.')

from interactive import enhanced_interactive_mode

enhanced_interactive_mode(
    _model="mistral-nemo:12b",
    csv_dir="../../csv_calls",
    results_dir="/content/saved_results",
    drive_path=None
)


╔══════════════════════════════════════════╗
║    🤖 АНАЛИТИК ЗВОНКОВ v3.0               ║
║    🚀 Google Drive Edition                ║
╚══════════════════════════════════════════╝
    
✅ Ollama подключен, модель: mistral-nemo:12b
📂 Загружаю данные из JSON файлов...
📂 Читаю данные из CSV файла: calls.csv
✅ Загружено 202 строк из CSV
✅ Преобразовано 202 записей звонков
📅 Диапазон дат: 01.10.2025 - 20.01.2026
🏷️  Уникальных тегов: 19
✅ Загружено 202 звонков

[Local] 📝 Вопрос (/? для помощи): на что клиенты чаще всего жаловались в октябре 2025
🔍 Анализирую: 'на что клиенты чаще всего жаловались в октябре 2025'

🔍 Анализирую запрос: 'на что клиенты чаще всего жаловались в октябре 2025'
🤖 Создаю план анализа...
   📅 Период: октябрь 2025
   🏷️  Теги: низкое_качество_стирки_или_чистки, не_заменили_ковры_вовремя, клиент_хочет_добавить_ковры, клиент_хочет_меньше_ковров, погашение_долга, расторжение_договора, возобновление_услуг, долго_нет_ответа_на_заявку, лишняя_доставка, доставили_не_те_ковры